# 予備実験

## 発話分類

In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np

In [4]:
import tensorflow_datasets as tfds
import tensorflow as tf

## データセット

In [5]:
train_dataset_path = "../Switchboard-Corpus/swda_data/train_set.txt"
test_dataset_path = "../Switchboard-Corpus/swda_data/test_set.txt"
val_dataset_path = "../Switchboard-Corpus/swda_data/val_set.txt"